In [ ]:
!pip install rpy2

  Using cached rpy2-3.4.5-py3-none-any.whl
  Using cached backports.zoneinfo-0.2.1-cp38-cp38-win_amd64.whl (38 kB)


In [ ]:
!pip install multiprocess

In [ ]:
from tqdm import tqdm

from multiprocess import Pool

import scipy

import networkx as nx
import random
import pandas as pd
import numpy as np

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

from sklearn.metrics.pairwise import cosine_similarity


from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")
import pickle
from scipy import stats

In [ ]:
%%writefile magic_functions.py
from tqdm import tqdm

from multiprocess import Pool

import scipy

import networkx as nx
import random
import pandas as pd
import numpy as np

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

from sklearn.metrics.pairwise import cosine_similarity


from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")
import pickle
from scipy import stats
### read trans probs
p_reg = pd.read_excel('C:/Users/Krishna/OneDrive/Documents/IIM_R1/proj2/prob_mats.xlsx', 'reg',index_col=0)
p_med = pd.read_excel('C:/Users/Krishna/OneDrive/Documents/IIM_R1/proj2/prob_mats.xlsx', 'med',index_col=0)
def create_network(N,nr,er,asa,bs_n,m_size):
    
    ### Graph generation
    ## Total organizations
    N=N
    ## region specific N
    n_regions_list=[int(0.46*N),int( 0.16*N),int( 0.38*N)]
    if (len(n_regions_list[0]*["NrA"]+n_regions_list[1]*["Eur"]+n_regions_list[2]*['Asia'])!=N):
        if (len(n_regions_list[0]*["NrA"]+n_regions_list[1]*["Eur"]+n_regions_list[2]*['Asia'])-N)>0:
            
            n_regions_list[0] = n_regions_list[0]+len(n_regions_list[0]*["NrA"]+n_regions_list[1]*["Eur"]+n_regions_list[2]*['Asia'])-N
        else:
            n_regions_list[0] = n_regions_list[0]-len(n_regions_list[0]*["NrA"]+n_regions_list[1]*["Eur"]+n_regions_list[2]*['Asia'])+N

            

    g = nx.random_partition_graph(n_regions_list, p_in= 0.60, p_out=0.15, seed=123, directed=True)

    edge_list_df=pd.DataFrame(list(g.edges(data=True)))
    edge_list_df.columns=['source','target','weight']
    
    
    ###
    #calculate n of b,bs,s
    nr_n=[int(nr[0]*n_regions_list[0]),int(nr[1]*n_regions_list[0]),int(nr[2]*n_regions_list[0])]
    er_n=[int(er[0]*n_regions_list[1]),int(er[1]*n_regions_list[1]),int(er[2]*n_regions_list[1])]
    asa_n=[int(asa[0]*n_regions_list[2]),int(asa[1]*n_regions_list[2]),int(asa[2]*n_regions_list[2])]

    if (np.sum(nr_n)<n_regions_list[0]):
        nr_n[0]=nr_n[0]+(n_regions_list[0]-np.sum(nr_n))
    if (np.sum(er_n)<n_regions_list[1]):
        er_n[0]=er_n[0]+(n_regions_list[1]-np.sum(er_n))
    if (np.sum(asa_n)<n_regions_list[2]):
        asa_n[0]=asa_n[0]+(n_regions_list[2]-np.sum(asa_n))
    ## if bs n controlled    
    k_diff=nr_n[2]-int((nr_n[0]+nr_n[2])/((nr_n[0]/nr_n[2])+1+bs_n))
    nr_n[2]=nr_n[2]-k_diff
    nr_n[0]=nr_n[0]+k_diff

    k_diff=er_n[2]-int((er_n[0]+er_n[2])/((er_n[0]/er_n[2])+1+bs_n))
    er_n[2]=er_n[2]-k_diff
    er_n[0]=er_n[0]+k_diff

    k_diff=asa_n[2]-int((asa_n[0]+asa_n[2])/((asa_n[0]/asa_n[2])+1+bs_n))
    asa_n[2]=asa_n[2]-k_diff
    asa_n[0]=asa_n[0]+k_diff    

    # choose b , s , bs
    #nr
    list1=range(0,n_regions_list[0])
    random.seed(10)
    list1_0=random.sample(list1, nr_n[0])
    random.seed(10)
    list1_1=random.sample(pd.DataFrame(set(list1)-set(list1_0)).iloc[:,0].tolist(),nr_n[1])
    random.seed(10)
    list1_2=random.sample(pd.DataFrame(set(list1)-(set(list1_1).union(set(list1_0)))).iloc[:,0].tolist(),nr_n[2])

    #eur
    list2=range(0+n_regions_list[0],n_regions_list[1]+n_regions_list[0])
    random.seed(10)
    list2_0=random.sample(list2, er_n[0])
    random.seed(10)
    list2_1=random.sample(pd.DataFrame(set(list2)-set(list2_0)).iloc[:,0].tolist(),er_n[1])
    random.seed(10)
    list2_2=random.sample(pd.DataFrame(set(list2)-(set(list2_1).union(set(list2_0)))).iloc[:,0].tolist(),er_n[2])

    #asi
    list3=range(0+n_regions_list[0]+n_regions_list[1],n_regions_list[2]+n_regions_list[0]+n_regions_list[1])
    random.seed(10)
    list3_0=random.sample(list3, asa_n[0])
    random.seed(10)
    list3_1=random.sample(pd.DataFrame(set(list3)-set(list3_0)).iloc[:,0].tolist(),asa_n[1])
    random.seed(10)
    list3_2=random.sample(pd.DataFrame(set(list3)-(set(list3_1).union(set(list3_0)))).iloc[:,0].tolist(),asa_n[2])

    #
    nodes_frame=pd.DataFrame(range(N),columns=['nodes'])
    
    nodes_frame['partition']=n_regions_list[0]*["NrA"]+n_regions_list[1]*["Eur"]+n_regions_list[2]*['Asia']

    nodes_frame['category']=""

    nodes_frame['category'][list1_0]="buyer"
    nodes_frame['category'][list2_0]="buyer"
    nodes_frame['category'][list3_0]="buyer"

    nodes_frame['category'][list1_1]="both"
    nodes_frame['category'][list2_1]="both"
    nodes_frame['category'][list3_1]="both"

    nodes_frame['category'][list1_2]="sup"
    nodes_frame['category'][list2_2]="sup"
    nodes_frame['category'][list3_2]="sup"
    
    #
    params_sn=pd.read_csv('skew_norm_params_reg_tier_mark_size.csv',index_col=0)
    nodes_frame['ms']=""
    ########### draw a market size based on reg and tier 
    for i in nodes_frame['nodes']:
        ps = params_sn.loc[(params_sn['tier']==nodes_frame['category'][i])&((params_sn['reg']==nodes_frame['partition'][i]))]
        #print(ps)
        np.random.seed(seed=123)
        nodes_frame['ms'][i]  = stats.skewnorm(ps['ae'], ps['loce'], ps['scalee']).rvs(1)[0]

    nqn1=np.quantile(nodes_frame['ms'],0.05)
    nqn3=np.quantile(nodes_frame['ms'],0.5)
    
    nodes_frame['ms']=nodes_frame['ms']+ m_size*nodes_frame['ms']
    
    dummy=pd.DataFrame(columns=['ms'])
    dummy['ms']=range(0,N)
    #dummy.head()
    #np.quantile(node_attr['ms'],0.67)
    for i in range(0,N):
        if nodes_frame.iloc[i,3]<=nqn1:
            dummy['ms'][i]="low"
        elif nodes_frame.iloc[i,3]<=nqn3:  
            dummy['ms'][i]="med"
        else:
            dummy['ms'][i]="high"
    #change_1        
    nodes_frame['ms2']=dummy['ms']

    

    buy_list=list1_0+list2_0+list3_0
    sup_list=list1_2+list2_2+list3_2

    edge_list_df_new=edge_list_df.drop([i for i, e in enumerate(list(edge_list_df['source'])) if e in set(sup_list)],axis=0)
    new_index=range(edge_list_df_new.shape[0])
    edge_list_df_new.index=new_index
    #edge_list_df_new.head()

    edge_list_df_new=edge_list_df_new.drop([i for i, e in enumerate(list(edge_list_df_new['target'])) if e in set(buy_list)],axis=0)
    new_index=range(edge_list_df_new.shape[0])
    edge_list_df_new.index=new_index
    
    g = nx.DiGraph( )
    # Add edges and edge attributes
    for i, elrow in edge_list_df_new.iterrows():
        g.add_edge(elrow[0], elrow[1], attr_dict=elrow[2])
    
    return [edge_list_df_new,nodes_frame,g]

####### To generate new attributes for desired state
####### To generate new attributes for desired state
"""def sample_lab_attr_all_init(N,m,sdvn):
    # Defining the R script and loading the instance in Python
    r = robjects.r
    r['source']('sampling_for_attributes_normal.R')
    # Loading the function we have defined in R.
    sampling_for_attributes_r2 = robjects.globalenv['sampling_for_attributes_normal']
    #Invoking the R function and getting the result
    df_result_r = sampling_for_attributes_r2(N,m,sdvn)
    #Converting it back to a pandas dataframe.
    df_result = pandas2ri.rpy2py(df_result_r)
    
    return(df_result)"""
def sample_lab_attr_all_init(N):
    # Defining the R script and loading the instance in Python
    r = robjects.r
    r['source']('sampling_for_attributes_normal.R')
    # Loading the function we have defined in R.
    sampling_for_attributes_r2 = robjects.globalenv['sampling_for_attributes_normal']
    #Invoking the R function and getting the result
    df_result_r = sampling_for_attributes_r2(N)
    #Converting it back to a pandas dataframe.
    df_result = pandas2ri.rpy2py(df_result_r)
    
    return(df_result)


####### To generate new attributes for desired state
def sample_lab_attr_new_B(N,reg,s_av1,s_av2):
    # Defining the R script and loading the instance in Python
    r = robjects.r
    r['source']('sampling_for_attributes.R')
    # Loading the function we have defined in R.
    sampling_for_attributes_r = robjects.globalenv['sampling_for_attributes']
    #Invoking the R function and getting the result
    df_result_r = sampling_for_attributes_r(N,reg)
    #print(df_result_r.head())
    #Converting it back to a pandas dataframe.
    df_result = pandas2ri.rpy2py(df_result_r)
    """if (s_av2-s_av1)<2:
        s_av2=np.min([s_av2+2,64.28571])
        if (s_av2-s_av1)>0:
            s_av1=np.max([s_av1-2,0])
        else:
            s_av1=np.max([s_av2-2,0])
            
        if s_av2>100:
            s_av2=100"""
                        
    sampled_data=df_result.loc[((df_result['score']>=(s_av1)) & (df_result['score']<=(s_av2)))]
    if sampled_data.shape[0]==0:
        s_av=np.mean([s_av1,s_av2])
        s_th=s_av*0.05
        sampled_data=df_result.loc[((df_result['score']>=(s_av-s_th)) | (df_result['score']<=(s_av+s_th)))]

        tmp_vector=np.abs(sampled_data['score']-s_av)
        #tmp_vector2=np.abs(df_result['score']-s_av)
        sampled_data=sampled_data.loc[tmp_vector==np.min(tmp_vector)]

        
    #tmp_vector=np.abs(sampled_data['score']-s_av)
    #tmp_vector2=np.abs(df_result['score']-s_av)
    #sampled_data=sampled_data.loc[tmp_vector==np.min(tmp_vector)]
    return(sampled_data.sample())

####### To generate new attributes for desired state
def sample_lab_attr_new(N,reg,s_av,s_th):
    # Defining the R script and loading the instance in Python
    r = robjects.r
    r['source']('sampling_for_attributes.R')
    # Loading the function we have defined in R.
    sampling_for_attributes_r = robjects.globalenv['sampling_for_attributes']
    #Invoking the R function and getting the result
    df_result_r = sampling_for_attributes_r(N,reg)
    #print(df_result_r.head())
    #Converting it back to a pandas dataframe.
    df_result = pandas2ri.rpy2py(df_result_r)
    sampled_data=df_result.loc[((df_result['score']>=(s_av-s_th)) | (df_result['score']<=(s_av+s_th)))]
    
    tmp_vector=np.abs(sampled_data['score']-s_av)
    #tmp_vector2=np.abs(df_result['score']-s_av)
    sampled_data=sampled_data.loc[tmp_vector==np.min(tmp_vector)]
    return(sampled_data)
#df_result.sample()
####### To generate new attributes for desired state
def sample_lab_attr_all(N,reg):
    
    # Defining the R script and loading the instance in Python
    r = robjects.r
    r['source']('sampling_for_attributes.R')
    # Loading the function we have defined in R.
    sampling_for_attributes_r = robjects.globalenv['sampling_for_attributes']
    #Invoking the R function and getting the result
    df_result_r = sampling_for_attributes_r(N,reg)
    #Converting it back to a pandas dataframe.
    df_result = pandas2ri.rpy2py(df_result_r)
    
    q1=np.quantile(df_result['score'],0.25)
    q3=np.quantile(df_result['score'],0.75)
    
    df_result['state']=""
    for i in range(0,df_result.shape[0]):
        if df_result['score'][i]<=q1:
            df_result['state'][i]= 'low' 
        elif df_result['score'][i]<=q3:
            df_result['state'][i]='med'
        else:
            df_result['state'][i]='high'
    #df_result=df_result.loc[df_result['state']==state_label]
    #tmp_vector=np.abs(df_result['score']-s_av)
    #sampled_data=df_result.loc[tmp_vector==np.min(tmp_vector)]
    #.sample()
    return(df_result)

def initial_random_attr(rows,random_p):    
    for i in range(0,rows):
        
        desired_state =random.choices(['high','med','low'],random_p)[0]
        sample_df_1=sample_lab_attr_org(desired_state , 500)
        if i==0:
            samples_df=sample_df_1
        else:
            samples_df=pd.concat([samples_df,sample_df_1],axis=0)
    return samples_df  

####### To generate new attributes for desired state
def sample_lab_attr_org(state_label,N):
    # Defining the R script and loading the instance in Python
    r = robjects.r
    r['source']('sampling_for_attributes.R')
    # Loading the function we have defined in R.
    sampling_for_attributes_r = robjects.globalenv['sampling_for_attributes']
    #Invoking the R function and getting the result
    df_result_r = sampling_for_attributes_r(N)
    #Converting it back to a pandas dataframe.
    df_result = pandas2ri.rpy2py(df_result_r)
    
    q1=np.quantile(df_result['score'],0.25)
    q3=np.quantile(df_result['score'],0.75)
    
    df_result['state']=""
    for i in range(0,df_result.shape[0]):
        if df_result['score'][i]<=q1:
            df_result['state'][i]= 'low' 
        elif df_result['score'][i]<=q3:
            df_result['state'][i]='med'
        else:
            df_result['state'][i]='high'
    df_result=df_result.loc[df_result['state']==state_label]
    #tmp_vector=np.abs(df_result['score']-s_av)
    #sampled_data=df_result.loc[tmp_vector==np.min(tmp_vector)]
    return(df_result.sample())



####### To generate new attributes for desired state
def prob_cdf(cur_sc,reg):
    # Defining the R script and loading the instance in Python
    r = robjects.r
    r['source']('prob_cdf.R')
    # Loading the function we have defined in R.
    prob_cdf_r = robjects.globalenv['prob_cdf']
    #Invoking the R function and getting the result
    df_result_p= prob_cdf_r(cur_sc,reg)
    #Converting it back to a pandas dataframe.
    #df_result = pandas2ri.rpy2py(df_result_r)
    
    return(df_result_p)
def prob_cdf_m(cur_sc,msh):
    # Defining the R script and loading the instance in Python
    r = robjects.r
    r['source']('prob_cdf.R')
    # Loading the function we have defined in R.
    prob_cdf_m = robjects.globalenv['prob_cdf_m']
    #Invoking the R function and getting the result
    df_result_p= prob_cdf_m(cur_sc,msh)
    #Converting it back to a pandas dataframe.
    #df_result = pandas2ri.rpy2py(df_result_r)
    
    return(df_result_p)
def prob_pdf(cur_sc,reg):
    # Defining the R script and loading the instance in Python
    r = robjects.r
    r['source']('prob_cdf.R')
    # Loading the function we have defined in R.
    prob_pdf_r = robjects.globalenv['prob_pdf']
    #Invoking the R function and getting the result
    df_result_p= prob_pdf_r(cur_sc,reg)
    #Converting it back to a pandas dataframe.
    #df_result = pandas2ri.rpy2py(df_result_r)
    
    return(df_result_p)
def prob_pdf_m(cur_sc,msh):
    # Defining the R script and loading the instance in Python
    r = robjects.r
    r['source']('prob_cdf.R')
    # Loading the function we have defined in R.
    prob_pdf_m = robjects.globalenv['prob_pdf_m']
    #Invoking the R function and getting the result
    df_result_p= prob_pdf_m(cur_sc,msh)
    #Converting it back to a pandas dataframe.
    #df_result = pandas2ri.rpy2py(df_result_r)
    
    return(df_result_p)

def simulation_continous (node_attr,edge_list_df,num_sim,w1,w2,w3,w4,w5,bs1,bs2,N,r_on,m_on,p_reg,p_med,probs_mat,probs_mat2,run_iter,alpha1,alpha2,alpha3):
    #nodes and edges
    N=N #number of org
    ncons=1000  #number of consumers
    g2=nx.random_partition_graph([N,ncons],0.5, 0.25)
    ed=pd.DataFrame(g2.edges())
    ed1=ed[(((ed[0]<N) & (ed[1]>N-1)) | ((ed[0]>N-1) & (ed[1]<N)))]
    ed1[1]-=N
    ed1.reset_index(inplace=True, drop=True)
    
    #partition = g.graph["partition"]
    #org=list(partition[0])
    #cons=list(partition[1])
   
    '''
    #for different weight proportion of organisations
    orgprop=[0.2,0.3,0.5]
    orgprop = [element * N for element in orgprop]
    orgprop = [int(x) for x in orgprop]
    orgwt=[0.25,0.5,0.7]
    cnt1=0
    ed1=pd.DataFrame()
    for i in range(len(orgprop)):
        g2=nx.random_partition_graph([orgprop[i], ncons], 0.5, orgwt[i])
        edg=pd.DataFrame(g2.edges())
        edg1=edg[(((edg[0]<orgprop[i]) & (edg[1]>orgprop[i]-1)) | ((edg[0]>orgprop[i]-1) & (edg[1]<orgprop[i])))]
        edg1[1]-=orgprop[i]
        edg1[0]+=cnt1
        cnt1+=orgprop[i]
        ed1=ed1.append(edg1)
    ed1.reset_index(inplace=True, drop=True)
    #ed1
    '''
    scons=np.array(pd.read_csv('Desktop/Data/final cons scores.csv')['x'])



    #add no. of consumer
    node_attr = node_attr
    edge_list_df = edge_list_df
    #P's    
    blanck_data_tot=np.empty([N,32,num_sim],dtype='object')
    #blanck_data_tot2=np.empty([N,4,num_sim],dtype='object')
# create initial edgelist of consumers and consumers
    for i in tqdm (range (num_sim), desc="Running i ..."): #no. of simulations
        blanck_data=np.empty([N,32],dtype='object')
        #blanck_data2=np.empty([N,4],dtype='object')


        # node attr to edge attr
        df_3=cosine_similarity(node_attr.iloc[:,:8])
        df_4=pd.DataFrame(df_3)
        df_4.values[[np.arange(len(df_4))]*2] = np.nan
        #mat_data.head()
        edge_list_2=df_4.stack().reset_index()
        edge_list_2.columns=['source','target','weight']
        #edge_list_2.head()
        edge_list_f=pd.merge(edge_list_df, edge_list_2,  how='left', left_on=['source','target'], right_on = ['source','target'])
        #edge_list_f.head()
        edge_list_f.drop('weight_x',axis=1,inplace=True)
        edge_list_f.columns=['source','target','weight']
        #edge_list_f.head()
        st = ["high","low"]
        
        ###########################################################
        for j in tqdm (range(0,N), desc="Running j..."): #looping through org
            #N=np.float(N)
            if len(list(np.where(edge_list_f.iloc[:,1]==j)[0]))>=1:
                ####################################################################################################    
                ########################################## MIMETIC##################################################
                ####################################################################################################
                st=["high","low"]
                st=pd.DataFrame(st)
                st.columns=['state']

                #Index in node attributes df['partitions'] == jth row partition column 
                p_tier_ind = [i for i, e in enumerate(list(node_attr['tier'])) if e in set([node_attr.iloc[j,10]])]
                t_node_attr = node_attr.iloc[p_tier_ind,:]
                #t_node_attr=t_node_attr.reset_index().iloc[:,1:]
                #t_node_attr.head()



                t_node_attr_score=t_node_attr['score'].copy()
                t_node_attr_score=t_node_attr_score.reset_index().iloc[:,1:]
                #t_node_attr_score

                #t_node_attr.index[tnr]

                for tnr in range(0,t_node_attr.shape[0]):
                    if node_attr.iloc[j,:]['score']<t_node_attr_score['score'][tnr]:
                        t_node_attr['state'][t_node_attr.index[tnr]]='high'
                    else:
                        t_node_attr['state'][t_node_attr.index[tnr]]='low'

                tier_p=pd.DataFrame(t_node_attr['state'].value_counts()/np.sum(t_node_attr['state'].value_counts()))
                tier_p=tier_p.reset_index()
                tier_p.columns=['state','t_p']
                #tier_p

                t_tier_p=pd.merge(st,tier_p,how="left",left_on=['state'],right_on='state')
                t_tier_p=t_tier_p.fillna(0.01)
                tier_p=t_tier_p
                #tier_p

                #d_tier.index

                #pd.DataFrame(node_attr.iloc[p_tier_ind,-2-2-1])
                #df_4.iloc[list(node_attr.iloc[p_tier_ind,:].index),j].reset_index().iloc[:,-1]

                #states and distances 
                #d_tier=pd.concat([node_attr.iloc[p_tier_ind,-2-2-1],
                #           df_4.iloc[list(node_attr.iloc[p_tier_ind,:].index),j] ],axis=1)
                d_tier=pd.concat([t_node_attr.iloc[:,-2-2-1],
                           df_4.iloc[list(node_attr.iloc[p_tier_ind,:].index),j] ],axis=1)

                #print(Ld)
                #d_tier=d_tier.drop([j])
                #d_tier=d_tier.reset_index()

                d_tier=d_tier.fillna(1)

                #and average disances per state
                d_tier_avg=d_tier.groupby(['state']).mean(str(j))
                #d_tier_avg



                s_tier_avg=pd.DataFrame(t_node_attr.groupby(['state']).mean()['score'])
                s_tier_avg=pd.merge(st,s_tier_avg, how='left', left_on=['state'], right_on = ['state']).fillna(node_attr.iloc[j,:]['score'])
                #s_tier_avg

                ## state local prob and avg distance
                mimetic_p=pd.merge(tier_p,d_tier_avg, how='left', left_on=['state'], right_on = ['state'])
                mimetic_p=pd.merge(mimetic_p,s_tier_avg, how='left', left_on=['state'], right_on = ['state'])

                #mimetic_p

                mimetic_p.columns=['state','tier_p','cur_node','score_m']
                mimetic_p['tier_p'] = mimetic_p['tier_p']/np.sum(mimetic_p['tier_p'])
                #mimetic_p

                #round(mimetic_p['score_m'][0])

                ################################################  regulatary mem
                region_ind = [i for i, e in enumerate(list(p_reg.columns)) if e in set([node_attr.iloc[j,9]])]
                ms_ind = [i for i, e in enumerate(list(p_med.columns)) if e in set([node_attr.iloc[j,12]])]

                h_reg=prob_pdf(round(round(mimetic_p['score_m'][0])),region_ind[0]+1)[0]/prob_pdf(round(node_attr.iloc[j,:]['score']),region_ind[0]+1)[0]
                l_reg=prob_pdf(round(round(mimetic_p['score_m'][1])),region_ind[0]+1)[0]/prob_pdf(round(node_attr.iloc[j,:]['score']),region_ind[0]+1)[0]
                pbreg=pd.DataFrame([h_reg/(h_reg+l_reg),l_reg/(h_reg+l_reg)],columns=['pbreg'])
                pbreg.index=['high','low']
                pbreg=pbreg.reset_index()
                pbreg.columns=['state','pbreg']
                #pbreg

                h_reg=prob_pdf_m(round(round(mimetic_p['score_m'][0])),ms_ind[0]+1)[0]/prob_pdf_m(round(node_attr.iloc[j,:]['score']),ms_ind[0]+1)[0]
                l_reg=prob_pdf_m(round(round(mimetic_p['score_m'][1])),ms_ind[0]+1)[0]/prob_pdf_m(round(node_attr.iloc[j,:]['score']),ms_ind[0]+1)[0]
                pbm=pd.DataFrame([h_reg/(h_reg+l_reg),l_reg/(h_reg+l_reg)],columns=['pbm'])
                pbm.index=['high','low']
                pbm=pbm.reset_index()
                pbm.columns=['state','pbm']
                #pbm
                pbreg.index=mimetic_p.index
                pbm.index=mimetic_p.index


                mimetic_p['pbreg_m']=pbreg['pbreg']
                mimetic_p['pbm_m']=pbm['pbm']
                #mimetic_p
                ####################################################################################################    
                ########################################## Local & Global / inform reg & normative #################
                ####################################################################################################
                #Index in node attributes df for rows with target column == j
                prnt_ind = [i for i, e in enumerate(list(node_attr.index)) if e in set(edge_list_f.loc[edge_list_f.iloc[:,1]==j].iloc[:,0])] 
                #Index in node attributes df for rows with target column == j
                prnt_ind2 = [i for i, e in enumerate(list(node_attr.index)) if e in set(edge_list_f.loc[edge_list_f.iloc[:,0]==j].iloc[:,1])] 

                l_node_attr = node_attr.iloc[prnt_ind,:]
                l_node_attr_score=l_node_attr['score'].copy()
                l_node_attr_score=l_node_attr_score.reset_index().iloc[:,1:]


                #len(l_node_attr.iloc[:,-2-2-1])

                #l_node_attr.loc[j]

                for tnr in range(0,l_node_attr.shape[0]):
                    if node_attr.iloc[j,:]['score']<l_node_attr_score['score'][tnr]:
                        l_node_attr['state'][l_node_attr.index[tnr]]='high'
                    else:
                        l_node_attr['state'][l_node_attr.index[tnr]]='low'

                l2_node_attr = node_attr.iloc[prnt_ind2,:]
                l2_node_attr_score=l2_node_attr['score'].copy()
                l2_node_attr_score=l2_node_attr_score.reset_index().iloc[:,1:]
                for tnr in range(0,l2_node_attr.shape[0]):
                    if node_attr.iloc[j,:]['score']<l2_node_attr_score['score'][tnr]:
                        l2_node_attr['state'][l2_node_attr.index[tnr]]='high'
                    else:
                        l2_node_attr['state'][l2_node_attr.index[tnr]]='low'



                #Lp1

                if len(prnt_ind2)>0:
                    #states prob of parent nodes(can also clculate d*count probabilities)
                    Lp1 = pd.DataFrame(l_node_attr.iloc[:,-2-2-1].value_counts()/np.sum(l_node_attr.iloc[:,-2-2-1].value_counts()))
                    Lp1 = Lp1.reset_index()
                    #states prob of parent nodes(can also clculate d*count probabilities)
                    Lp2 = pd.DataFrame(l2_node_attr.iloc[:,-2-2-1].value_counts()/np.sum(l2_node_attr.iloc[:,-2-2-1].value_counts()))
                    Lp2 = Lp2.reset_index()
                    Lp1=pd.merge(st,Lp1,how="left",left_on=['state'],right_on='index').fillna(0.01)
                    Lp2=pd.merge(st,Lp2,how="left",left_on=['state'],right_on='index').fillna(0.01)
                    Lp=pd.merge(Lp1,Lp2,how="left",left_on=['state_x'],right_on='state_x')
                    #print(Lp.head())
                    Lp['state']=bs1*Lp['state_y_x']+bs2*Lp['state_y_y']
                    Lp=Lp.iloc[:,[0,5]]
                    Lp.columns=['index','state']
                    #print(Lp1.head())
                    #print(Lp2.head())


                else:
                    #states prob of parent nodes(can also clculate d*count probabilities)
                    Lp = pd.DataFrame(l_node_attr.iloc[:,-2-2-1].value_counts()/np.sum(l_node_attr.iloc[:,-2-2-1].value_counts()))
                    Lp = Lp.reset_index()
                    #print(Lp)
                    Lp=pd.merge(st,Lp,how="left",left_on=['state'],right_on='index').fillna(0.01)
                    Lp=Lp.iloc[:,[0,2]]
                    #print(Lp)
                    Lp.columns=['index','state']                

                    #Lp.head()

                if len(prnt_ind2)>0:

                    #states and distances 
                    Ld1=pd.concat([l_node_attr.iloc[:,-2-2-1],
                           df_4.iloc[list(node_attr.iloc[prnt_ind,:].index),j] ],axis=1)

                    Lad1=Ld1.groupby(['state']).mean()

                    #states and distances 
                    Ld2=pd.concat([l2_node_attr.iloc[:,-2-2-1],
                           df_4.iloc[list(node_attr.iloc[prnt_ind2,:].index),j] ],axis=1)
                    #Lp2.head()
                    Lad2=Ld2.groupby(['state']).mean()
                    Lad1=pd.merge(st,Lad1,how="left",left_on=['state'],right_on='state').fillna(0.01)
                    Lad2=pd.merge(st,Lad2,how="left",left_on=['state'],right_on='state').fillna(0.01)
                    Lad=pd.merge(Lad1,Lad2,how="left",left_on=['state'],right_on='state').fillna(0.01)
                    #print(Lad)
                    Lad['state_n']=bs1*Lad[str(j)+'_x']+bs2*Lad[str(j)+'_y']
                    Lad=Lad.iloc[:,[0,3]]
                    Lad.columns=['state',str(j)]
                    Lad.index=Lad['state']
                    Lad=Lad.iloc[:,1]
                    #print(Lad.head())
                    s_l1_avg=pd.DataFrame(l_node_attr.groupby(['state']).mean()['score'])
                    s_l1_avg=pd.merge(st,s_l1_avg, how='left', left_on=['state'], right_on = ['state']).fillna(node_attr.iloc[j,:]['score'])
                    s_l2_avg=pd.DataFrame(l2_node_attr.groupby(['state']).mean()['score'])
                    s_l2_avg=pd.merge(st,s_l2_avg, how='left', left_on=['state'], right_on = ['state']).fillna(node_attr.iloc[j,:]['score'])
                    s_l_avg=pd.merge(s_l1_avg,s_l2_avg,how="left",left_on=['state'],right_on='state')
                    #print(s_l_avg)
                    s_l_avg['score_n']=bs1*s_l_avg['score'+'_x']+bs2*s_l_avg['score'+'_y']
                    s_l_avg=s_l_avg.iloc[:,[0,3]]
                    s_l_avg.columns=['state','score']

                else:
                    #states and distances 
                    Ld=pd.concat([l_node_attr.iloc[:,-2-2-1],
                               df_4.iloc[list(node_attr.iloc[prnt_ind,:].index),j] ],axis=1)
                    #print(Ld)
                    #and average disances per state
                    Lad=Ld.groupby(['state']).mean()#str(j)
                    Lad=pd.merge(st,Lad,how="left",left_on=['state'],right_on='state').fillna(0.01)

                    Lad=Lad.reset_index()

                    s_l_avg=pd.DataFrame(l_node_attr.groupby(['state']).mean()['score'])
                    s_l_avg=pd.merge(st,s_l_avg, how='left', left_on=['state'], right_on = ['state']).fillna(node_attr.iloc[j,:]['score'])
                    s_l_avg=s_l_avg.reset_index()

                    #Lad.head()
                #print(Lad)

                #Lad

                #s_l_avg

                #print(dist_local)
                if len(prnt_ind2)>0:

                    ## state local prob and avg distance
                    dist_local=pd.merge(Lp,Lad, how='left', left_on=['index'], right_on = ['state'])
                    dist_local.columns=['state','local_prob','cur_node_l']
                    #dist_local

                    dist_local=pd.merge(dist_local,s_l_avg, how='left', left_on=['state'], right_on = ['state'])
                else :
                    #bs1*s_l_avg['score'+'_x']+s_l_avg*Lad['score'+'_y']
                    dist_local=pd.merge(Lp,Lad, how='left', left_on=['index'], right_on = ['state'])
                    dist_local=dist_local.iloc[:,[0,1,4]]
                    dist_local.columns=['state','local_prob','cur_node_l']
                    #dist_local
                    #print(s_l_avg)

                    dist_local=pd.merge(dist_local,s_l_avg, how='left', left_on=['state'], right_on = ['state'])
                    dist_local=dist_local.iloc[:,[0,1,2,4]]



                #dist_local

                dist_local.columns=['state','local_prob','cur_node_l','score_l']

                dist_local['local_prob']=dist_local['local_prob']/np.sum(dist_local['local_prob'])

                #print(dist_local)

                h_reg=prob_pdf(round(round(dist_local['score_l'][0])),region_ind[0]+1)[0]/prob_pdf(round(node_attr.iloc[j,:]['score']),region_ind[0]+1)[0]
                l_reg=prob_pdf(round(round(dist_local['score_l'][1])),region_ind[0]+1)[0]/prob_pdf(round(node_attr.iloc[j,:]['score']),region_ind[0]+1)[0]
                pbreg=pd.DataFrame([h_reg/(h_reg+l_reg),l_reg/(h_reg+l_reg)],columns=['pbreg'])
                pbreg.index=['high','low']
                pbreg=pbreg.reset_index()
                pbreg.columns=['state','pbreg']
                #pbreg

                h_reg=prob_pdf_m(round(round(dist_local['score_l'][0])),ms_ind[0]+1)[0]/prob_pdf_m(round(node_attr.iloc[j,:]['score']),ms_ind[0]+1)[0]
                l_reg=prob_pdf_m(round(round(dist_local['score_l'][1])),ms_ind[0]+1)[0]/prob_pdf_m(round(node_attr.iloc[j,:]['score']),ms_ind[0]+1)[0]
                pbm=pd.DataFrame([h_reg/(h_reg+l_reg),l_reg/(h_reg+l_reg)],columns=['pbm'])
                pbm.index=['high','low']
                pbm=pbm.reset_index()
                pbm.columns=['state','pbm']
                #pbm
                pbreg.index=mimetic_p.index
                pbm.index=mimetic_p.index


                dist_local['pbreg_l']=pbreg['pbreg']
                dist_local['pbm_l']=pbm['pbm']
                #dist_local

                ## global prob
                #glb_p=pd.DataFrame(node_attr['state'].value_counts()/np.sum(node_attr['state'].value_counts()))
                #glb_p=glb_p.reset_index()
                #glb_p.columns=['state','g_p']
                st=["high","low"]
                st=pd.DataFrame(st)
                st.columns=['state']
                #Index in node attributes df['partitions'] == jth row partition column 
                p_region_ind = [i for i, e in enumerate(list(node_attr['partition'])) if e in set([node_attr.iloc[j,9]])]
                r_node_attr = node_attr.iloc[p_region_ind,:]

                r_node_attr_score=r_node_attr['score'].copy()
                r_node_attr_score=r_node_attr_score.reset_index().iloc[:,1:]
                for tnr in range(0,r_node_attr.shape[0]):
                    if node_attr.iloc[j,:]['score']<r_node_attr_score['score'][tnr]:
                        r_node_attr['state'][r_node_attr.index[tnr]]='high'
                    else:
                        r_node_attr['state'][r_node_attr.index[tnr]]='low'




                glb_p=pd.DataFrame(r_node_attr['state'].value_counts()/np.sum(r_node_attr['state'].value_counts()))


                glb_p=glb_p.reset_index()
                glb_p.columns=['state','g_p']

                t_glb_p=pd.merge(st,glb_p,how="left",left_on=['state'],right_on='state')
                t_glb_p=t_glb_p.fillna(0.01)
                glb_p=t_glb_p

                #print(glb_p)
                #states and distances 
                gd=pd.concat([r_node_attr.iloc[:,-2-2-1],
                           df_4.iloc[list(node_attr.iloc[p_region_ind,:].index),j] ],axis=1)
                #print(gd)

                #and average disances per state
                gad=gd.groupby(['state']).mean(str(j))
                gad=pd.merge(st,gad,how="left",left_on=['state'],right_on='state')
                #gad.reset_index(inplace=True)
                #print(gad)
                s_g_avg=pd.DataFrame(r_node_attr.groupby(['state']).mean()['score'])
                s_g_avg=pd.merge(st,s_g_avg, how='left', left_on=['state'], right_on = ['state']).fillna(node_attr.iloc[j,:]['score'])
                #s_g_avg

                ## state local prob and avg distance
                dist_global=pd.merge(glb_p,gad, how='left', left_on=['state'], right_on = ['state'])
                dist_global=pd.merge(dist_global,s_g_avg, how='left', left_on=['state'], right_on = ['state'])

                #dist_local

                dist_global.columns=['state','glob_prob','cur_node_g','score_g']

                dist_global['glob_prob'] =dist_global['glob_prob']/np.sum(dist_global['glob_prob'])
                #print(dist_global)

                h_reg=prob_pdf(round(round(dist_global['score_g'][0])),region_ind[0]+1)[0]/prob_pdf(round(node_attr.iloc[j,:]['score']),region_ind[0]+1)[0]
                l_reg=prob_pdf(round(round(dist_global['score_g'][1])),region_ind[0]+1)[0]/prob_pdf(round(node_attr.iloc[j,:]['score']),region_ind[0]+1)[0]
                pbreg=pd.DataFrame([h_reg/(h_reg+l_reg),l_reg/(h_reg+l_reg)],columns=['pbreg'])
                pbreg.index=['high','low']
                pbreg=pbreg.reset_index()
                pbreg.columns=['state','pbreg']
                #pbreg

                h_reg=prob_pdf_m(round(round(dist_global['score_g'][0])),ms_ind[0]+1)[0]/prob_pdf_m(round(node_attr.iloc[j,:]['score']),ms_ind[0]+1)[0]
                l_reg=prob_pdf_m(round(round(dist_global['score_g'][1])),ms_ind[0]+1)[0]/prob_pdf_m(round(node_attr.iloc[j,:]['score']),ms_ind[0]+1)[0]
                pbm=pd.DataFrame([h_reg/(h_reg+l_reg),l_reg/(h_reg+l_reg)],columns=['pbm'])
                pbm.index=['high','low']
                pbm=pbm.reset_index()
                pbm.columns=['state','pbm']
                #pbm
                pbreg.index=mimetic_p.index
                pbm.index=mimetic_p.index


                dist_global['pbreg_g']=pbreg['pbreg']
                dist_global['pbm_g']=pbm['pbm']
                #dist_global

                #print('glb_p')
                if (((i+1)*(j+1)) % 5000) ==0: print(dist_global)
                ## all memetic
                dist_local_global=pd.merge(dist_global,dist_local, how='left', left_on=['state'], right_on = ['state'])
                dist_local_global=dist_local_global.fillna(0.01)

                #dist_local_global['m_p']=dist_local_global.product(axis=1)/np.sum(dist_local_global.product(axis=1))
                #print(dist_local_global)
                #
                ####################################################################################################    
                ########################################## All_ Pressures ##########################################
                ####################################################################################################
                #
                # All presures
                all_p = pd.merge(mimetic_p,dist_local_global,how='left', left_on=['state'], right_on = ['state'])
                all_p=all_p.fillna(0.01)
                #all_p = pd.merge(all_p,mimetic_p,how='left', left_on=['state'], right_on = ['state'])
                #all_p 
                #= all_p.iloc[:,[0,4,5,6]]

                #0.25*all_p.iloc[:,3:5].product(axis=1)

                #all_p.iloc[:,3:5]

                #w1=w2=w3=w4=0.25
                #all_p

                #w1*all_p['tier_p'][0]*all_p['cur_node'][0]*all_p['score_m'][0]

                #w1=w2=w3=w4=0.25

                all_p_tpd=all_p.copy()
                #all_p_tpd
                all_p_new=pd.DataFrame(all_p_tpd['state'])
                #print(all_p_new)
                all_p_new['tier_p']=(all_p_tpd['tier_p']*all_p_tpd['cur_node'])/np.sum(all_p_tpd['tier_p']*all_p_tpd['cur_node'])
                all_p_new['glob_prob']=(all_p_tpd['glob_prob']*all_p_tpd['cur_node_g'])/np.sum(all_p_tpd['glob_prob']*all_p_tpd['cur_node_g'])
                all_p_new['local_prob']=(all_p_tpd['local_prob']*all_p_tpd['cur_node_l'])/np.sum(all_p_tpd['local_prob']*all_p_tpd['cur_node_l'])
                all_p_new['pbreg']=(all_p_tpd['pbreg_m']+all_p_tpd['pbreg_g']+all_p_tpd['pbreg_l'])/np.sum(all_p_tpd['pbreg_m']+all_p_tpd['pbreg_g']+all_p_tpd['pbreg_l'])
                all_p_new['pbm']=(all_p_tpd['pbm_m']+all_p_tpd['pbm_g']+all_p_tpd['pbm_l'])/np.sum(all_p_tpd['pbm_m']+all_p_tpd['pbm_g']+all_p_tpd['pbm_l'])
                all_p_new['score_m']=all_p_tpd['score_m']
                all_p_new['score_l']=all_p_tpd['score_l']
                all_p_new['score_g']=all_p_tpd['score_g']

                #pd.DataFrame(all_p_new)
                all_p=all_p_new


                """
                if r_on==1:        
                    rpbr =[all_p['pbreg_m'][0],all_p['pbreg_l'][0],all_p['pbreg_g'][0]]
                else:
                    rpbr =[1,1,1]
                if m_on==1:        
                    mpbm =[all_p['pbm_m'][0],all_p['pbm_l'][0],all_p['pbm_g'][0]]
                else:
                    mpbm =[1,1,1]

                ptotalh=np.exp((w1*all_p['tier_p'][0]*all_p['cur_node'][0]*rpbr[0]*mpbm[0]*all_p['score_m'][0]+w2*all_p['local_prob'][0]*all_p['cur_node_l'][0]*rpbr[1]*mpbm[1]*all_p['score_l'][0]+w3*all_p['glob_prob'][0]*all_p['cur_node_g'][0]*rpbr[2]*mpbm[2]*all_p['score_g'][0])/w4)/(1+np.exp((w1*all_p['tier_p'][0]*all_p['cur_node'][0]*rpbr[0]*mpbm[0]*all_p['score_m'][0]+w2*all_p['local_prob'][0]*all_p['cur_node_l'][0]*rpbr[1]*mpbm[1]*all_p['score_l'][0]+w3*all_p['glob_prob'][0]*all_p['cur_node_g'][0]*rpbr[2]*mpbm[2]*all_p['score_g'][0])/w4))
                ptotall=1-ptotalh
                ptot=pd.DataFrame([ptotalh,ptotall],columns=['ptotal'])
                ptot.index=all_p.index
                all_p['ptotal']=ptot['ptotal']
                #all_p
                """
                if r_on==1:        
                    rpbr =all_p['pbreg'][0]
                else:
                    rpbr =0
                if m_on==1:        
                    mpbm =all_p['pbm'][0]
                else:
                    mpbm =0
                rpmp=(all_p['pbreg']*all_p['pbm'])/np.sum(all_p['pbreg']*all_p['pbm'])
                if r_on==0:
                    rpmp[0]=1
                    rpmp[1]=1
                    
                #no. of consumers connected to this org in proportion
                #in ptotalh, add w4*object
                #same addition in ptotall
                #j is looping through org
                c=np.array(ed1[ed1[0]==j][1]) #d
                sum1=0
                for cnt in range(len(c)):           
                    sum1+=scons[c[cnt]]  #where is scons
                avg1=sum1/len(c)
                ptotalh=((np.exp(w1*(all_p['tier_p'][0]+alpha1)+w2*(all_p['local_prob'][0]+alpha2)+w3*(all_p['glob_prob'][0]+alpha3)+w4*avg1)/(1+np.exp(w1*(all_p['tier_p'][0]+alpha1)+w2*(all_p['local_prob'][0]+alpha2)+w3*(all_p['glob_prob'][0]+alpha3)+w4*avg1)))*(rpmp[0]))
                #ptotalh=np.exp((w1*all_p['tier_p'][0]+w2*all_p['local_prob'][0]+w3*all_p['glob_prob'][0]+w4*rpbr+w5*mpbm))/(1+np.exp((w1*all_p['tier_p'][0]+w2*all_p['local_prob'][0]+w3*all_p['glob_prob'][0]+w4*rpbr+w5*mpbm)))
                #ptotalh=((np.exp(w1*(all_p['tier_p'][0]+alpha1)+w2*(all_p['local_prob'][0]+alpha2)+w3*(all_p['glob_prob'][0]+alpha3))/(1+np.exp(w1*(all_p['tier_p'][0]+alpha1)+w2*(all_p['local_prob'][0]+alpha2)+w3*(all_p['glob_prob'][0]+alpha3))))*(rpmp[0]))
                #ptotalh=ptotalh/np.sum(ptotalh)

                #ptotall=1-ptotalh
                ptotall=(1/(1+np.exp(w1*(all_p['tier_p'][0]+alpha1)+w2*(all_p['local_prob'][0]+alpha2)+w3*(all_p['glob_prob'][0]+alpha3))))*(rpmp[1])
                ptot=pd.DataFrame([ptotalh,ptotall],columns=['ptotal'])
                ptot.index=all_p.index
                all_p['ptotal']=ptot['ptotal']
                all_p['ptotal']=all_p['ptotal']/np.sum(all_p['ptotal'])
                #all_p
                for k in range(ncons):
                    cons_org=(ed1[0].value_counts()[j])/ncons  #number of consumers connected to org/ncons
                    cons_org*=100
                    diff=scons[k]-sample_df_1.values[0][7]
                     #number of consumers connected to the organisation
                    p2=(np.exp(0.5*diff+0.5*cons_org))/(1+np.exp(0.5*diff+0.5*cons_org))
                    if(p2>0.5):
                        if((len(ed1[(ed1[0]==j) & (ed1[1]==k)]))==0):  #if edge not found
                            ed1=ed1.append({0:j, 1:k}, ignore_index=True)  #then make the edge
                    elif(p2==0.5):
                        if(random.random()>0.5):
                            if((len(ed1[(ed1[0]==j) & (ed1[1]==k)]))==0):
                                ed1=ed1.append({0:j, 1:k}, ignore_index=True)
                        else:
                            if((len(ed1[(ed1[0]==j) & (ed1[1]==k)]))>0):
                                ed1=ed1.drop(ed1[((ed1[0] == j) & ( ed1[1] == k))].index)
                    else:
                        if((len(ed1[(ed1[0]==j) & (ed1[1]==k)]))>0):  #if edge found
                            ed1=ed1.drop(ed1[((ed1[0] == j) & ( ed1[1] == k))].index)  #remove the edge
                ''''
                ed1.drop(ed1[ed1[0]==j].index, inplace=True)  
                for cnt in range(ncons):
                    if(sorg[j]>scons[cnt]):
                        ed1=ed1.append({0:j, 1:cnt}, ignore_index=True)
                    elif(sorg[j]==scons[cnt]):
                        if(random.random()>0.5):
                            ed1=ed1.append({0:j, 1:cnt}, ignore_index=True)
                            '''
    #edsave.append(np.array(ed1)) 

                #print(all_p)

                #0.6224593312018546
                """
                d_s_ind=np.where(all_p['ptotal']==np.max(all_p['ptotal']))[0][0]
                """
                if np.count_nonzero([w1,w2,w3])!=0:
                    if all_p['ptotal'][0]>0.6224593312018546:
                        d_s_ind=0
                    elif all_p['ptotal'][0]<0.6224593312018546:
                        d_s_ind=1
                    else:
                        d_s_ind = 1 if np.random.random()<0.5 else 0
                else:
                    if all_p['ptotal'][0]>0.5:
                        d_s_ind=0
                    elif all_p['ptotal'][0]<0.5:
                        d_s_ind=1
                    else:
                        d_s_ind = 1 if np.random.random()<0.5 else 0


                #print(d_s_ind)
                """                                                                                                            
                if r_on==1:        
                    rpbr =[all_p['pbreg_m'][d_s_ind],all_p['pbreg_l'][d_s_ind],all_p['pbreg_g'][d_s_ind]]
                else:
                    rpbr =[1,1,1]
                if m_on==1:        
                    mpbm =[all_p['pbm_m'][d_s_ind],all_p['pbm_l'][d_s_ind],all_p['pbm_g'][d_s_ind]]
                else:
                    mpbm =[1,1,1]
                """
                if r_on==1:        
                    rpbr =all_p['pbreg'][d_s_ind]
                else:
                    rpbr =0
                if m_on==1:        
                    mpbm =all_p['pbm'][d_s_ind]
                else:
                    mpbm =0

                """s_av=(w1*all_p['tier_p'][d_s_ind]*all_p['cur_node'][d_s_ind]*rpbr[0]*mpbm[0]*all_p['score_m'][d_s_ind]+w2*all_p['local_prob'][d_s_ind]*all_p['cur_node_l'][d_s_ind]*rpbr[1]*mpbm[1]*all_p['score_l'][d_s_ind]+w3*all_p['glob_prob'][d_s_ind]*all_p['cur_node_g'][d_s_ind]*rpbr[2]*mpbm[2]*all_p['score_g'][d_s_ind])/(w1*all_p['tier_p'][d_s_ind]*all_p['cur_node'][d_s_ind]*rpbr[0]*mpbm[0]+w2*all_p['local_prob'][d_s_ind]*all_p['cur_node_l'][d_s_ind]*rpbr[1]*mpbm[1]+w3*all_p['glob_prob'][d_s_ind]*all_p['cur_node_g'][d_s_ind]*rpbr[2]*mpbm[2])"""
                """s_av=(w1*all_p['score_m'][d_s_ind]+w2*all_p['score_l'][d_s_ind]+w3*all_p['score_g'][d_s_ind])/(w1+w2+w3)"""
                #    s_av1=np.min([all_p['score_m'][d_s_ind],all_p['score_l'][d_s_ind],all_p['score_g'][d_s_ind]])
                #    s_av2=np.max([all_p['score_m'][d_s_ind],all_p['score_l'][d_s_ind],all_p['score_g'][d_s_ind]])

                if w1==0:
                    s_av1=np.min([all_p['score_l'][d_s_ind],all_p['score_g'][d_s_ind]])
                    s_av2=np.max([all_p['score_l'][d_s_ind],all_p['score_g'][d_s_ind]])
                elif w2==0:
                    s_av1=np.min([all_p['score_m'][d_s_ind],all_p['score_g'][d_s_ind]])
                    s_av2=np.max([all_p['score_m'][d_s_ind],all_p['score_g'][d_s_ind]])
                elif w3==0:
                    s_av1=np.min([all_p['score_m'][d_s_ind],all_p['score_l'][d_s_ind]])
                    s_av2=np.max([all_p['score_m'][d_s_ind],all_p['score_l'][d_s_ind]])
                elif np.count_nonzero([w1,w2,w3])==1:
                    s_av1=[w1*all_p['score_m'][d_s_ind]+w2*all_p['score_l'][d_s_ind]+w3*all_p['score_g'][d_s_ind]]
                    s_av2=[w1*all_p['score_m'][d_s_ind]+w2*all_p['score_l'][d_s_ind]+w3*all_p['score_g'][d_s_ind]]
                elif np.count_nonzero([w1,w2,w3])==0:
                    s_av1=node_attr['score'][j]
                    s_av2=node_attr['score'][j]
                else:
                    
                    s_av1=np.min([all_p['score_m'][d_s_ind],all_p['score_l'][d_s_ind],all_p['score_g'][d_s_ind]])
                    s_av2=np.max([all_p['score_m'][d_s_ind],all_p['score_l'][d_s_ind],all_p['score_g'][d_s_ind]])

                    





                #s_av

                #region_ind

                #print(all_p)
                probs_mat[i,j]=np.max(all_p['ptotal'])
                if i==0:
                    probs_mat2[i,j]=np.max(all_p['ptotal'])
                else:
                    probs_mat2[i+j,:]=probs_mat[i,j]

                ## hihest prob label
                #desired_state = random.choices(list(all_p['state']),list(all_p['all']))[0]
                #desired_state = all_p['state'][d_s_ind] 
                #desired_state
                #desired_state = list(all_p.loc[all_p['all']==np.max(all_p['all'])]['state'])[0]



                ##### draw attributes with given label

                """sample_df_1=sample_lab_attr_new(np.float(N),region_ind[0],s_av,0.05*s_av)"""
                """if s_av1==s_av2:
                    if d_s_ind==0:
                    
                        sample_df_1=sample_lab_attr_new_B(np.float(N),region_ind[0],s_av1+0.1,100)
                    else:
                        sample_df_1=sample_lab_attr_new_B(np.float(N),region_ind[0],0,s_av2+0.12)
                else:
                    sample_df_1=sample_lab_attr_new_B(np.float(N),region_ind[0],s_av1+0.1,s_av2+0.12)"""
                if s_av1==s_av2:
                    if d_s_ind==0:
                    
                        sample_df_1=sample_lab_attr_new_B(np.float(N),region_ind[0],s_av1+0.1,100)
                    else:
                        sample_df_1=sample_lab_attr_new_B(np.float(N),region_ind[0],0,s_av2+0.12)
                else:
                    if d_s_ind==0:
                    
                        sample_df_1=sample_lab_attr_new_B(np.float(N),region_ind[0],(s_av1+s_av2)/2,100)
                    else:
                        sample_df_1=sample_lab_attr_new_B(np.float(N),region_ind[0],0,(s_av1+s_av2)/2)
                    
                    
                
                
                ####################################################################################################    
                ########################################## Update  attributes ######################################
                ####################################################################################################
                ## update node attributes 
                for k,replc in enumerate(sample_df_1.values[0]):
                    node_attr.iloc[j,k]=replc                    
                ## update edge attributes
                # node attr to edge attr
                df_3=cosine_similarity(node_attr.iloc[:,:8])
                df_4=pd.DataFrame(df_3)
                df_4.values[[np.arange(len(df_4))]*2] = np.nan
                #mat_data.head()
                edge_list_2=df_4.stack().reset_index()
                edge_list_2.columns=['source','target','weight']
                #edge_list_2.head()
                edge_list_f=pd.merge(edge_list_df, edge_list_2,  how='left', left_on=['source','target'], right_on = ['source','target'])
                #edge_list_f.head()
                edge_list_f.drop('weight_x',axis=1,inplace=True)
                edge_list_f.columns=['source','target','weight']



                for k,replc in enumerate(node_attr.iloc[j,:].values):
                    blanck_data[j,k]=replc 

                for k,replc in enumerate(all_p.iloc[0,1:].values):
                    blanck_data[j,k+13]=replc 
                blanck_data[j,29]=j
                blanck_data[j,30]=i
                blanck_data[j,31]=all_p['state'][d_s_ind]


                #blanck_data2[:,:2,i]=np.array(edge_list_f) 



            else:

                ####2
                ####################################################################################################    
                ########################################## MIMETIC##################################################
                ####################################################################################################
                st=["high","low"]
                st=pd.DataFrame(st)
                st.columns=['state']

                #Index in node attributes df['partitions'] == jth row partition column 
                p_tier_ind = [i for i, e in enumerate(list(node_attr['tier'])) if e in set([node_attr.iloc[j,10]])]
                t_node_attr = node_attr.iloc[p_tier_ind,:]
                #t_node_attr=t_node_attr.reset_index().iloc[:,1:]
                #t_node_attr.head()



                t_node_attr_score=t_node_attr['score'].copy()
                t_node_attr_score=t_node_attr_score.reset_index().iloc[:,1:]
                #t_node_attr_score

                #t_node_attr.index[tnr]

                for tnr in range(0,t_node_attr.shape[0]):
                    if node_attr.iloc[j,:]['score']<t_node_attr_score['score'][tnr]:
                        t_node_attr['state'][t_node_attr.index[tnr]]='high'
                    else:
                        t_node_attr['state'][t_node_attr.index[tnr]]='low'

                tier_p=pd.DataFrame(t_node_attr['state'].value_counts()/np.sum(t_node_attr['state'].value_counts()))
                tier_p=tier_p.reset_index()
                tier_p.columns=['state','t_p']
                #tier_p

                t_tier_p=pd.merge(st,tier_p,how="left",left_on=['state'],right_on='state')
                t_tier_p=t_tier_p.fillna(0.01)
                tier_p=t_tier_p
                #tier_p

                #d_tier.index

                #pd.DataFrame(node_attr.iloc[p_tier_ind,-2-2-1])
                #df_4.iloc[list(node_attr.iloc[p_tier_ind,:].index),j].reset_index().iloc[:,-1]

                #states and distances 
                #d_tier=pd.concat([node_attr.iloc[p_tier_ind,-2-2-1],
                #           df_4.iloc[list(node_attr.iloc[p_tier_ind,:].index),j] ],axis=1)
                d_tier=pd.concat([t_node_attr.iloc[:,-2-2-1],
                           df_4.iloc[list(node_attr.iloc[p_tier_ind,:].index),j] ],axis=1)

                #print(Ld)
                #d_tier=d_tier.drop([j])
                #d_tier=d_tier.reset_index()

                d_tier=d_tier.fillna(1)

                #and average disances per state
                d_tier_avg=d_tier.groupby(['state']).mean(str(j))
                #d_tier_avg



                s_tier_avg=pd.DataFrame(t_node_attr.groupby(['state']).mean()['score'])
                s_tier_avg=pd.merge(st,s_tier_avg, how='left', left_on=['state'], right_on = ['state']).fillna(node_attr.iloc[j,:]['score'])
                #s_tier_avg

                ## state local prob and avg distance
                mimetic_p=pd.merge(tier_p,d_tier_avg, how='left', left_on=['state'], right_on = ['state'])
                mimetic_p=pd.merge(mimetic_p,s_tier_avg, how='left', left_on=['state'], right_on = ['state'])

                #mimetic_p

                mimetic_p.columns=['state','tier_p','cur_node','score_m']
                mimetic_p['tier_p'] = mimetic_p['tier_p']/np.sum(mimetic_p['tier_p'])
                #mimetic_p

                #round(mimetic_p['score_m'][0])

                ################################################  regulatary mem
                region_ind = [i for i, e in enumerate(list(p_reg.columns)) if e in set([node_attr.iloc[j,9]])]
                ms_ind = [i for i, e in enumerate(list(p_med.columns)) if e in set([node_attr.iloc[j,12]])]

                h_reg=prob_pdf(round(round(mimetic_p['score_m'][0])),region_ind[0]+1)[0]/prob_pdf(round(node_attr.iloc[j,:]['score']),region_ind[0]+1)[0]
                l_reg=prob_pdf(round(round(mimetic_p['score_m'][1])),region_ind[0]+1)[0]/prob_pdf(round(node_attr.iloc[j,:]['score']),region_ind[0]+1)[0]
                pbreg=pd.DataFrame([h_reg/(h_reg+l_reg),l_reg/(h_reg+l_reg)],columns=['pbreg'])
                pbreg.index=['high','low']
                pbreg=pbreg.reset_index()
                pbreg.columns=['state','pbreg']
                #pbreg

                h_reg=prob_pdf_m(round(round(mimetic_p['score_m'][0])),ms_ind[0]+1)[0]/prob_pdf_m(round(node_attr.iloc[j,:]['score']),ms_ind[0]+1)[0]
                l_reg=prob_pdf_m(round(round(mimetic_p['score_m'][1])),ms_ind[0]+1)[0]/prob_pdf_m(round(node_attr.iloc[j,:]['score']),ms_ind[0]+1)[0]
                pbm=pd.DataFrame([h_reg/(h_reg+l_reg),l_reg/(h_reg+l_reg)],columns=['pbm'])
                pbm.index=['high','low']
                pbm=pbm.reset_index()
                pbm.columns=['state','pbm']
                #pbm
                pbreg.index=mimetic_p.index
                pbm.index=mimetic_p.index


                mimetic_p['pbreg_m']=pbreg['pbreg']
                mimetic_p['pbm_m']=pbm['pbm']
                #mimetic_p
                ####################################################################################################    
                ########################################## Local & Global / inform reg & normative #################
                ####################################################################################################
                #Index in node attributes df for rows with target column == j
                """prnt_ind = [i for i, e in enumerate(list(node_attr.index)) if e in set(edge_list_f.loc[edge_list_f.iloc[:,1]==j].iloc[:,0])]""" 
                #Index in node attributes df for rows with target column == j
                prnt_ind2 = [i for i, e in enumerate(list(node_attr.index)) if e in set(edge_list_f.loc[edge_list_f.iloc[:,0]==j].iloc[:,1])] 

                """l_node_attr = node_attr.iloc[prnt_ind,:]
                l_node_attr_score=l_node_attr['score'].copy()
                l_node_attr_score=l_node_attr_score.reset_index().iloc[:,1:]


                #len(l_node_attr.iloc[:,-2-2-1])

                #l_node_attr.loc[j]

                for tnr in range(0,l_node_attr.shape[0]):
                    if node_attr.iloc[j,:]['score']<l_node_attr_score['score'][tnr]:
                        l_node_attr['state'][l_node_attr.index[tnr]]='high'
                    else:
                        l_node_attr['state'][l_node_attr.index[tnr]]='low'"""

                l2_node_attr = node_attr.iloc[prnt_ind2,:]
                l2_node_attr_score=l2_node_attr['score'].copy()
                l2_node_attr_score=l2_node_attr_score.reset_index().iloc[:,1:]
                for tnr in range(0,l2_node_attr.shape[0]):
                    if node_attr.iloc[j,:]['score']<l2_node_attr_score['score'][tnr]:
                        l2_node_attr['state'][l2_node_attr.index[tnr]]='high'
                    else:
                        l2_node_attr['state'][l2_node_attr.index[tnr]]='low'



                #Lp1

                """if len(prnt_ind2)>0:
                    #states prob of parent nodes(can also clculate d*count probabilities)
                    Lp1 = pd.DataFrame(l_node_attr.iloc[:,-2-2-1].value_counts()/np.sum(l_node_attr.iloc[:,-2-2-1].value_counts()))
                    Lp1 = Lp1.reset_index()
                    #states prob of parent nodes(can also clculate d*count probabilities)
                    Lp2 = pd.DataFrame(l2_node_attr.iloc[:,-2-2-1].value_counts()/np.sum(l2_node_attr.iloc[:,-2-2-1].value_counts()))
                    Lp2 = Lp2.reset_index()
                    Lp1=pd.merge(st,Lp1,how="left",left_on=['state'],right_on='index').fillna(0.01)
                    Lp2=pd.merge(st,Lp2,how="left",left_on=['state'],right_on='index').fillna(0.01)
                    Lp=pd.merge(Lp1,Lp2,how="left",left_on=['state_x'],right_on='state_x')
                    #print(Lp.head())
                    Lp['state']=bs1*Lp['state_y_x']+bs2*Lp['state_y_y']
                    Lp=Lp.iloc[:,[0,5]]
                    Lp.columns=['index','state']
                    #print(Lp1.head())
                    #print(Lp2.head())


                else:"""
                #states prob of parent nodes(can also clculate d*count probabilities)
                Lp = pd.DataFrame(l2_node_attr.iloc[:,-2-2-1].value_counts()/np.sum(l2_node_attr.iloc[:,-2-2-1].value_counts()))
                Lp = Lp.reset_index()
                #print(Lp)
                Lp=pd.merge(st,Lp,how="left",left_on=['state'],right_on='index').fillna(0.01)
                Lp=Lp.iloc[:,[0,2]]
                #print(Lp)
                Lp.columns=['index','state']                
                #print(Lp)
                

                
                    #Lp.head()

                """if len(prnt_ind2)>0:

                    #states and distances 
                    Ld1=pd.concat([l_node_attr.iloc[:,-2-2-1],
                           df_4.iloc[list(node_attr.iloc[prnt_ind,:].index),j] ],axis=1)

                    Lad1=Ld1.groupby(['state']).mean()

                    #states and distances 
                    Ld2=pd.concat([l2_node_attr.iloc[:,-2-2-1],
                           df_4.iloc[list(node_attr.iloc[prnt_ind2,:].index),j] ],axis=1)
                    #Lp2.head()
                    Lad2=Ld2.groupby(['state']).mean()
                    Lad1=pd.merge(st,Lad1,how="left",left_on=['state'],right_on='state').fillna(0.01)
                    Lad2=pd.merge(st,Lad2,how="left",left_on=['state'],right_on='state').fillna(0.01)
                    Lad=pd.merge(Lad1,Lad2,how="left",left_on=['state'],right_on='state').fillna(0.01)
                    #print(Lad)
                    Lad['state_n']=bs1*Lad[str(j)+'_x']+bs2*Lad[str(j)+'_y']
                    Lad=Lad.iloc[:,[0,3]]
                    Lad.columns=['state',str(j)]
                    #print(Lad.head())
                    s_l1_avg=pd.DataFrame(l_node_attr.groupby(['state']).mean()['score'])
                    s_l1_avg=pd.merge(st,s_l1_avg, how='left', left_on=['state'], right_on = ['state']).fillna(node_attr.iloc[j,:]['score'])
                    s_l2_avg=pd.DataFrame(l2_node_attr.groupby(['state']).mean()['score'])
                    s_l2_avg=pd.merge(st,s_l2_avg, how='left', left_on=['state'], right_on = ['state']).fillna(node_attr.iloc[j,:]['score'])
                    s_l_avg=pd.merge(s_l1_avg,s_l2_avg,how="left",left_on=['state'],right_on='state')
                    #print(s_l_avg)
                    s_l_avg['score_n']=bs1*s_l_avg['score'+'_x']+bs2*s_l_avg['score'+'_y']
                    s_l_avg=s_l_avg.iloc[:,[0,3]]
                    s_l_avg.columns=['state','score']

                else:"""
                #states and distances 
                Ld=pd.concat([l2_node_attr.iloc[:,-2-2-1],
                           df_4.iloc[list(node_attr.iloc[prnt_ind2,:].index),j] ],axis=1)
                #print(Ld)
                #and average disances per state
                Lad=Ld.groupby(['state']).mean()#str(j)
                #print(Lad)
                Lad=pd.merge(st,Lad,how="left",left_on=['state'],right_on='state').fillna(0.01)
                #print(Lad)

                Lad=Lad.reset_index()
                #print(Lad)

                s_l_avg=pd.DataFrame(l2_node_attr.groupby(['state']).mean()['score'])
                s_l_avg=pd.merge(st,s_l_avg, how='left', left_on=['state'], right_on = ['state']).fillna(node_attr.iloc[j,:]['score'])
                s_l_avg=s_l_avg.reset_index()

                    #Lad.head()
                #print(Lad)
                #Lad.index=Lad['state']
                #Lad=Lad.iloc[:,1:]
                #print(Lad)
                #Lad

                #s_l_avg

                #bs1*s_l_avg['score'+'_x']+s_l_avg*Lad['score'+'_y']

                ## state local prob and avg distance
                
                dist_local=pd.merge(Lp,Lad, how='left', left_on=['index'], right_on = ['state'])
                dist_local=dist_local.iloc[:,[0,1,4]]
                dist_local.columns=['state','local_prob','cur_node_l']
                #dist_local
                #print(s_l_avg)

                dist_local=pd.merge(dist_local,s_l_avg, how='left', left_on=['state'], right_on = ['state'])
                dist_local=dist_local.iloc[:,[0,1,2,4]]
                #print(dist_local)

                #dist_local=dist_local.drop(['index'])

                #dist_local

                dist_local.columns=['state','local_prob','cur_node_l','score_l']

                dist_local['local_prob']=dist_local['local_prob']/np.sum(dist_local['local_prob'])

                #print(dist_local)

                h_reg=prob_pdf(round(round(dist_local['score_l'][0])),region_ind[0]+1)[0]/prob_pdf(round(node_attr.iloc[j,:]['score']),region_ind[0]+1)[0]
                l_reg=prob_pdf(round(round(dist_local['score_l'][1])),region_ind[0]+1)[0]/prob_pdf(round(node_attr.iloc[j,:]['score']),region_ind[0]+1)[0]
                pbreg=pd.DataFrame([h_reg/(h_reg+l_reg),l_reg/(h_reg+l_reg)],columns=['pbreg'])
                pbreg.index=['high','low']
                pbreg=pbreg.reset_index()
                pbreg.columns=['state','pbreg']
                #pbreg

                h_reg=prob_pdf_m(round(round(dist_local['score_l'][0])),ms_ind[0]+1)[0]/prob_pdf_m(round(node_attr.iloc[j,:]['score']),ms_ind[0]+1)[0]
                l_reg=prob_pdf_m(round(round(dist_local['score_l'][1])),ms_ind[0]+1)[0]/prob_pdf_m(round(node_attr.iloc[j,:]['score']),ms_ind[0]+1)[0]
                pbm=pd.DataFrame([h_reg/(h_reg+l_reg),l_reg/(h_reg+l_reg)],columns=['pbm'])
                pbm.index=['high','low']
                pbm=pbm.reset_index()
                pbm.columns=['state','pbm']
                #pbm
                pbreg.index=mimetic_p.index
                pbm.index=mimetic_p.index


                dist_local['pbreg_l']=pbreg['pbreg']
                dist_local['pbm_l']=pbm['pbm']
                #dist_local

                ## global prob
                #glb_p=pd.DataFrame(node_attr['state'].value_counts()/np.sum(node_attr['state'].value_counts()))
                #glb_p=glb_p.reset_index()
                #glb_p.columns=['state','g_p']
                st=["high","low"]
                st=pd.DataFrame(st)
                st.columns=['state']
                #Index in node attributes df['partitions'] == jth row partition column 
                p_region_ind = [i for i, e in enumerate(list(node_attr['partition'])) if e in set([node_attr.iloc[j,9]])]
                r_node_attr = node_attr.iloc[p_region_ind,:]

                r_node_attr_score=r_node_attr['score'].copy()
                r_node_attr_score=r_node_attr_score.reset_index().iloc[:,1:]
                for tnr in range(0,r_node_attr.shape[0]):
                    if node_attr.iloc[j,:]['score']<r_node_attr_score['score'][tnr]:
                        r_node_attr['state'][r_node_attr.index[tnr]]='high'
                    else:
                        r_node_attr['state'][r_node_attr.index[tnr]]='low'




                glb_p=pd.DataFrame(r_node_attr['state'].value_counts()/np.sum(r_node_attr['state'].value_counts()))


                glb_p=glb_p.reset_index()
                glb_p.columns=['state','g_p']

                t_glb_p=pd.merge(st,glb_p,how="left",left_on=['state'],right_on='state')
                t_glb_p=t_glb_p.fillna(0.01)
                glb_p=t_glb_p

                #print(glb_p)
                #states and distances 
                gd=pd.concat([r_node_attr.iloc[:,-2-2-1],
                           df_4.iloc[list(node_attr.iloc[p_region_ind,:].index),j] ],axis=1)
                #print(gd)

                #and average disances per state
                gad=gd.groupby(['state']).mean(str(j))
                gad=pd.merge(st,gad,how="left",left_on=['state'],right_on='state')
                #gad.reset_index(inplace=True)
                #print(gad)
                s_g_avg=pd.DataFrame(r_node_attr.groupby(['state']).mean()['score'])
                s_g_avg=pd.merge(st,s_g_avg, how='left', left_on=['state'], right_on = ['state']).fillna(node_attr.iloc[j,:]['score'])
                #s_g_avg

                ## state local prob and avg distance
                dist_global=pd.merge(glb_p,gad, how='left', left_on=['state'], right_on = ['state'])
                dist_global=pd.merge(dist_global,s_g_avg, how='left', left_on=['state'], right_on = ['state'])

                #dist_local

                dist_global.columns=['state','glob_prob','cur_node_g','score_g']

                dist_global['glob_prob'] =dist_global['glob_prob']/np.sum(dist_global['glob_prob'])
                #print(dist_global)

                h_reg=prob_pdf(round(round(dist_global['score_g'][0])),region_ind[0]+1)[0]/prob_pdf(round(node_attr.iloc[j,:]['score']),region_ind[0]+1)[0]
                l_reg=prob_pdf(round(round(dist_global['score_g'][1])),region_ind[0]+1)[0]/prob_pdf(round(node_attr.iloc[j,:]['score']),region_ind[0]+1)[0]
                pbreg=pd.DataFrame([h_reg/(h_reg+l_reg),l_reg/(h_reg+l_reg)],columns=['pbreg'])
                pbreg.index=['high','low']
                pbreg=pbreg.reset_index()
                pbreg.columns=['state','pbreg']
                #pbreg

                h_reg=prob_pdf_m(round(round(dist_global['score_g'][0])),ms_ind[0]+1)[0]/prob_pdf_m(round(node_attr.iloc[j,:]['score']),ms_ind[0]+1)[0]
                l_reg=prob_pdf_m(round(round(dist_global['score_g'][1])),ms_ind[0]+1)[0]/prob_pdf_m(round(node_attr.iloc[j,:]['score']),ms_ind[0]+1)[0]
                pbm=pd.DataFrame([h_reg/(h_reg+l_reg),l_reg/(h_reg+l_reg)],columns=['pbm'])
                pbm.index=['high','low']
                pbm=pbm.reset_index()
                pbm.columns=['state','pbm']
                #pbm
                pbreg.index=mimetic_p.index
                pbm.index=mimetic_p.index


                dist_global['pbreg_g']=pbreg['pbreg']
                dist_global['pbm_g']=pbm['pbm']
                #dist_global

                #print('glb_p')
                #if (((i+1)*(j+1)) % 5000) ==0: print(dist_global)
                ## all memetic
                dist_local_global=pd.merge(dist_global,dist_local, how='left', left_on=['state'], right_on = ['state'])
                dist_local_global=dist_local_global.fillna(0.01)

                #dist_local_global['m_p']=dist_local_global.product(axis=1)/np.sum(dist_local_global.product(axis=1))
                #print(dist_local_global)
                #
                ####################################################################################################    
                ########################################## All_ Pressures ##########################################
                ####################################################################################################
                #
                # All presures
                all_p = pd.merge(mimetic_p,dist_local_global,how='left', left_on=['state'], right_on = ['state'])
                all_p=all_p.fillna(0.01)
                #all_p = pd.merge(all_p,mimetic_p,how='left', left_on=['state'], right_on = ['state'])
                #all_p 
                #= all_p.iloc[:,[0,4,5,6]]

                #0.25*all_p.iloc[:,3:5].product(axis=1)

                #all_p.iloc[:,3:5]

                #w1=w2=w3=w4=0.25
                #all_p

                #w1*all_p['tier_p'][0]*all_p['cur_node'][0]*all_p['score_m'][0]

                #w1=w2=w3=w4=0.25
                all_p_tpd=all_p.copy()
                #all_p_tpd
                all_p_new=pd.DataFrame(all_p_tpd['state'])
                #print(all_p_new)
                all_p_new['tier_p']=(all_p_tpd['tier_p']*all_p_tpd['cur_node'])/np.sum(all_p_tpd['tier_p']*all_p_tpd['cur_node'])
                all_p_new['glob_prob']=(all_p_tpd['glob_prob']*all_p_tpd['cur_node_g'])/np.sum(all_p_tpd['glob_prob']*all_p_tpd['cur_node_g'])
                all_p_new['local_prob']=(all_p_tpd['local_prob']*all_p_tpd['cur_node_l'])/np.sum(all_p_tpd['local_prob']*all_p_tpd['cur_node_l'])
                all_p_new['pbreg']=(all_p_tpd['pbreg_m']+all_p_tpd['pbreg_g']+all_p_tpd['pbreg_l'])/np.sum(all_p_tpd['pbreg_m']+all_p_tpd['pbreg_g']+all_p_tpd['pbreg_l'])
                all_p_new['pbm']=(all_p_tpd['pbm_m']+all_p_tpd['pbm_g']+all_p_tpd['pbm_l'])/np.sum(all_p_tpd['pbm_m']+all_p_tpd['pbm_g']+all_p_tpd['pbm_l'])
                all_p_new['score_m']=all_p_tpd['score_m']
                all_p_new['score_l']=all_p_tpd['score_l']
                all_p_new['score_g']=all_p_tpd['score_g']

                #pd.DataFrame(all_p_new)
                all_p=all_p_new


                """
                if r_on==1:        
                    rpbr =[all_p['pbreg_m'][0],all_p['pbreg_l'][0],all_p['pbreg_g'][0]]
                else:
                    rpbr =[1,1,1]
                if m_on==1:        
                    mpbm =[all_p['pbm_m'][0],all_p['pbm_l'][0],all_p['pbm_g'][0]]
                else:
                    mpbm =[1,1,1]

                ptotalh=np.exp((w1*all_p['tier_p'][0]*all_p['cur_node'][0]*rpbr[0]*mpbm[0]*all_p['score_m'][0]+w2*all_p['local_prob'][0]*all_p['cur_node_l'][0]*rpbr[1]*mpbm[1]*all_p['score_l'][0]+w3*all_p['glob_prob'][0]*all_p['cur_node_g'][0]*rpbr[2]*mpbm[2]*all_p['score_g'][0])/w4)/(1+np.exp((w1*all_p['tier_p'][0]*all_p['cur_node'][0]*rpbr[0]*mpbm[0]*all_p['score_m'][0]+w2*all_p['local_prob'][0]*all_p['cur_node_l'][0]*rpbr[1]*mpbm[1]*all_p['score_l'][0]+w3*all_p['glob_prob'][0]*all_p['cur_node_g'][0]*rpbr[2]*mpbm[2]*all_p['score_g'][0])/w4))
                ptotall=1-ptotalh
                ptot=pd.DataFrame([ptotalh,ptotall],columns=['ptotal'])
                ptot.index=all_p.index
                all_p['ptotal']=ptot['ptotal']
                #all_p
                """
                if r_on==1:        
                    rpbr =all_p['pbreg'][0]
                else:
                    rpbr =0
                if m_on==1:        
                    mpbm =all_p['pbm'][0]
                else:
                    mpbm =0
                rpmp=(all_p['pbreg']*all_p['pbm'])/np.sum(all_p['pbreg']*all_p['pbm'])
                if r_on==0:
                    rpmp[0]=1
                    rpmp[1]=1
                

                #ptotalh=np.exp((w1*all_p['tier_p'][0]+w2*all_p['local_prob'][0]+w3*all_p['glob_prob'][0]+w4*rpbr+w5*mpbm))/(1+np.exp((w1*all_p['tier_p'][0]+w2*all_p['local_prob'][0]+w3*all_p['glob_prob'][0]+w4*rpbr+w5*mpbm)))
                ptotalh=((np.exp(w1*(all_p['tier_p'][0]+alpha1)+w2*(all_p['local_prob'][0]+alpha2)+w3*(all_p['glob_prob'][0]+alpha3))/(1+np.exp(w1*(all_p['tier_p'][0]+alpha1)+w2*(all_p['local_prob'][0]+alpha2)+w3*(all_p['glob_prob'][0]+alpha3))))*(rpmp[0]))
                #ptotalh=ptotalh/np.sum(ptotalh)

                #ptotall=1-ptotalh
                ptotall=(1/(1+np.exp(w1*(all_p['tier_p'][0]+alpha1)+w2*(all_p['local_prob'][0]+alpha2)+w3*(all_p['glob_prob'][0]+alpha3))))*(rpmp[1])
                ptot=pd.DataFrame([ptotalh,ptotall],columns=['ptotal'])
                ptot.index=all_p.index
                all_p['ptotal']=ptot['ptotal']
                all_p['ptotal']=all_p['ptotal']/np.sum(all_p['ptotal'])
                #all_p


                #print(all_p)

                #0.6224593312018546
                """
                d_s_ind=np.where(all_p['ptotal']==np.max(all_p['ptotal']))[0][0]
                """
                if np.count_nonzero([w1,w2,w3])!=0:
                    if all_p['ptotal'][0]>0.6224593312018546:
                        d_s_ind=0
                    elif all_p['ptotal'][0]<0.6224593312018546:
                        d_s_ind=1
                    else:
                        d_s_ind = 1 if np.random.random()<0.5 else 0
                else:
                    if all_p['ptotal'][0]>0.5:
                        d_s_ind=0
                    elif all_p['ptotal'][0]<0.5:
                        d_s_ind=1
                    else:
                        d_s_ind = 1 if np.random.random()<0.5 else 0


                #print(d_s_ind)
                """                                                                                                            
                if r_on==1:        
                    rpbr =[all_p['pbreg_m'][d_s_ind],all_p['pbreg_l'][d_s_ind],all_p['pbreg_g'][d_s_ind]]
                else:
                    rpbr =[1,1,1]
                if m_on==1:        
                    mpbm =[all_p['pbm_m'][d_s_ind],all_p['pbm_l'][d_s_ind],all_p['pbm_g'][d_s_ind]]
                else:
                    mpbm =[1,1,1]
                """
                if r_on==1:        
                    rpbr =all_p['pbreg'][d_s_ind]
                else:
                    rpbr =0
                if m_on==1:        
                    mpbm =all_p['pbm'][d_s_ind]
                else:
                    mpbm =0

                """s_av=(w1*all_p['tier_p'][d_s_ind]*all_p['cur_node'][d_s_ind]*rpbr[0]*mpbm[0]*all_p['score_m'][d_s_ind]+w2*all_p['local_prob'][d_s_ind]*all_p['cur_node_l'][d_s_ind]*rpbr[1]*mpbm[1]*all_p['score_l'][d_s_ind]+w3*all_p['glob_prob'][d_s_ind]*all_p['cur_node_g'][d_s_ind]*rpbr[2]*mpbm[2]*all_p['score_g'][d_s_ind])/(w1*all_p['tier_p'][d_s_ind]*all_p['cur_node'][d_s_ind]*rpbr[0]*mpbm[0]+w2*all_p['local_prob'][d_s_ind]*all_p['cur_node_l'][d_s_ind]*rpbr[1]*mpbm[1]+w3*all_p['glob_prob'][d_s_ind]*all_p['cur_node_g'][d_s_ind]*rpbr[2]*mpbm[2])"""
                """s_av=(w1*all_p['score_m'][d_s_ind]+w2*all_p['score_l'][d_s_ind]+w3*all_p['score_g'][d_s_ind])/(w1+w2+w3)"""
                #    s_av1=np.min([all_p['score_m'][d_s_ind],all_p['score_l'][d_s_ind],all_p['score_g'][d_s_ind]])
                #    s_av2=np.max([all_p['score_m'][d_s_ind],all_p['score_l'][d_s_ind],all_p['score_g'][d_s_ind]])

                if w1==0:
                    s_av1=np.min([all_p['score_l'][d_s_ind],all_p['score_g'][d_s_ind]])
                    s_av2=np.max([all_p['score_l'][d_s_ind],all_p['score_g'][d_s_ind]])
                elif w2==0:
                    s_av1=np.min([all_p['score_m'][d_s_ind],all_p['score_g'][d_s_ind]])
                    s_av2=np.max([all_p['score_m'][d_s_ind],all_p['score_g'][d_s_ind]])
                elif w3==0:
                    s_av1=np.min([all_p['score_m'][d_s_ind],all_p['score_l'][d_s_ind]])
                    s_av2=np.max([all_p['score_m'][d_s_ind],all_p['score_l'][d_s_ind]])
                elif np.count_nonzero([w1,w2,w3])==1:
                    s_av1=[w1*all_p['score_m'][d_s_ind]+w2*all_p['score_l'][d_s_ind]+w3*all_p['score_g'][d_s_ind]]
                    s_av2=[w1*all_p['score_m'][d_s_ind]+w2*all_p['score_l'][d_s_ind]+w3*all_p['score_g'][d_s_ind]]
                elif np.count_nonzero([w1,w2,w3])==0:
                    s_av1=node_attr['score'][j]
                    s_av2=node_attr['score'][j]
                else:
                    
                    s_av1=np.min([all_p['score_m'][d_s_ind],all_p['score_l'][d_s_ind],all_p['score_g'][d_s_ind]])
                    s_av2=np.max([all_p['score_m'][d_s_ind],all_p['score_l'][d_s_ind],all_p['score_g'][d_s_ind]])





                #s_av

                #region_ind

                #print(all_p)
                probs_mat[i,j]=np.max(all_p['ptotal'])
                if i==0:
                    probs_mat2[i,j]=np.max(all_p['ptotal'])
                else:
                    probs_mat2[i+j,:]=probs_mat[i,j]

                ## hihest prob label
                #desired_state = random.choices(list(all_p['state']),list(all_p['all']))[0]
                #desired_state = all_p['state'][d_s_ind] 
                #desired_state
                #desired_state = list(all_p.loc[all_p['all']==np.max(all_p['all'])]['state'])[0]



                ##### draw attributes with given label

                """sample_df_1=sample_lab_attr_new(np.float(N),region_ind[0],s_av,0.05*s_av)"""
                """if s_av1==s_av2:
                    if d_s_ind==0:
                    
                        sample_df_1=sample_lab_attr_new_B(np.float(N),region_ind[0],s_av1+0.1,100)
                    else:
                        sample_df_1=sample_lab_attr_new_B(np.float(N),region_ind[0],0,s_av2+0.12)
                else:
                    sample_df_1=sample_lab_attr_new_B(np.float(N),region_ind[0],s_av1+0.1,s_av2+0.12)"""
                if s_av1==s_av2:
                    if d_s_ind==0:
                    
                        sample_df_1=sample_lab_attr_new_B(np.float(N),region_ind[0],s_av1+0.1,100)
                    else:
                        sample_df_1=sample_lab_attr_new_B(np.float(N),region_ind[0],0,s_av2+0.12)
                else:
                    if d_s_ind==0:
                    
                        sample_df_1=sample_lab_attr_new_B(np.float(N),region_ind[0],(s_av1+s_av2)/2,100)
                    else:
                        sample_df_1=sample_lab_attr_new_B(np.float(N),region_ind[0],0,(s_av1+s_av2)/2)
                    
                    
                
                ####################################################################################################    
                ########################################## Update  attributes ######################################
                ####################################################################################################
                ## update node attributes 
                for k,replc in enumerate(sample_df_1.values[0]):
                    node_attr.iloc[j,k]=replc                    
                ## update edge attributes
                # node attr to edge attr
                df_3=cosine_similarity(node_attr.iloc[:,:8])
                df_4=pd.DataFrame(df_3)
                df_4.values[[np.arange(len(df_4))]*2] = np.nan
                #mat_data.head()
                edge_list_2=df_4.stack().reset_index()
                edge_list_2.columns=['source','target','weight']
                #edge_list_2.head()
                edge_list_f=pd.merge(edge_list_df, edge_list_2,  how='left', left_on=['source','target'], right_on = ['source','target'])
                #edge_list_f.head()
                edge_list_f.drop('weight_x',axis=1,inplace=True)
                edge_list_f.columns=['source','target','weight']



                for k,replc in enumerate(node_attr.iloc[j,:].values):
                    blanck_data[j,k]=replc 

                for k,replc in enumerate(all_p.iloc[0,1:].values):
                    blanck_data[j,k+13]=replc 
                blanck_data[j,29]=j
                blanck_data[j,30]=i
                blanck_data[j,31]=all_p['state'][d_s_ind] 

                #blanck_data2[:,:2,i]=np.array(edge_list_f) 

        blanck_data_tot[:,:,i]=pd.DataFrame(blanck_data)

                

#if i>= 2:
    #if i%5==0:
        #probs_mat_pr.append(np.prod(np.log(probs_mat[i,:]),axis=1))
        
        edge_list_f.to_csv(folder_location+"sc_"+str(run_iter+1)+"/"+str(i)+"_edge_attr.csv")
    reshaped_bd = np.vstack(blanck_data_tot[:,:,i] for i in range(num_sim))
    reshaped_bd_df=pd.DataFrame(reshaped_bd)
    reshaped_bd_df.to_csv(folder_location+"sc_"+str(run_iter+1)+"/"+ "other_node_attr.csv")

            
    print('Complete')
import os
os.getcwd()

#scenarios
scn_params=pd.read_csv('test_scenarios_norm_sense_u_l.csv')

scn_params.shape

folder_location='simulation_log/testscenarios_norm_parallel_1000/'
#folder_location='simulation_log/'

def process_func(run_iter):
    #print('@@@@@ run iter @@@@@ --' + str(run_iter))
    nr=[0.22,0.35,0.43]
    er=[0.38,0.13,0.50]
    asa=[0.22,0.06,0.72]
   
    N=scn_params.iloc[run_iter,0]
    bs_n=scn_params.iloc[run_iter,3]
    m_size=scn_params.iloc[run_iter,4]
    bs1=scn_params.iloc[run_iter,1]
    bs2=scn_params.iloc[run_iter,2]
    
    ################################################### create network
    network_created=create_network(N,nr,er,asa,bs_n,m_size)
    #graph
    g=network_created[2]
    #centrality
    deg_cent = nx.degree_centrality(g)
    in_deg_cent = nx.in_degree_centrality(g)
    out_deg_cent = nx.out_degree_centrality(g)
    eigen_cent = nx.eigenvector_centrality(g)
    #katz_cent = nx.katz_centrality(g)
    closeness_cent = nx.closeness_centrality(g)
    #betw_cent = nx.betweenness_centrality(g)
    #vote_cent = nx.voterank(g)
    deg=pd.DataFrame(list(deg_cent.values()),columns=['deg'])
    indeg=pd.DataFrame(list(in_deg_cent.values()),columns=['indeg'])
    outdeg=pd.DataFrame(list(out_deg_cent.values()),columns=['outdeg'])
    eigencent=pd.DataFrame(list(eigen_cent.values()),columns=['eigdeg'])
    closeness=pd.DataFrame(list(closeness_cent.values()),columns=['closedeg'])
    all_net_p=pd.concat([deg,indeg,outdeg,closeness,eigencent],axis=1)
    #tier and ms
    nodes_frame=network_created[1]
    #edge list
    edge_list_df_new=network_created[0]
    edge_list_df=edge_list_df_new.copy()


    n_regions_list=[int(0.46*N),int( 0.16*N),int( 0.38*N)]
    if (len(n_regions_list[0]*["NrA"]+n_regions_list[1]*["Eur"]+n_regions_list[2]*['Asia'])!=N):
        if (len(n_regions_list[0]*["NrA"]+n_regions_list[1]*["Eur"]+n_regions_list[2]*['Asia'])-N)>0:

            n_regions_list[0] = n_regions_list[0]+len(n_regions_list[0]*["NrA"]+n_regions_list[1]*["Eur"]+n_regions_list[2]*['Asia'])-N
        else:
            n_regions_list[0] = n_regions_list[0]-len(n_regions_list[0]*["NrA"]+n_regions_list[1]*["Eur"]+n_regions_list[2]*['Asia'])+N

    #print(n_regions_list)
    #till partition
    ###################################################initial attributes at random one at a time
    #### method 1
    

    """init_samples = sample_lab_attr_all_init(np.float(N),30,10)"""
    init_samples = sample_lab_attr_all_init(np.float(N))
    #### method 2
    """init_samples1 = initial_random_attr(n_regions_list[0],np.array([0.2,0.3,0.5]))
    init_samples1=init_samples1.reset_index().iloc[:,1:]
    init_samples2 = initial_random_attr(n_regions_list[1],np.array([0.3,0.3,0.4]))
    init_samples2=init_samples2.reset_index().iloc[:,1:]
    init_samples3 = initial_random_attr(n_regions_list[2],np.array([0.5,0.3,0.2]))
    init_samples3=init_samples3.reset_index().iloc[:,1:]

    init_samples=pd.concat([init_samples1,init_samples2,init_samples3],axis=0)"""
    
    ### method 3
    """init_samples1 = sample_lab_attr_all(n_regions_list[0],1)
    init_samples1=init_samples1.reset_index().iloc[:,1:]
    init_samples2 = sample_lab_attr_all(n_regions_list[1],2)
    init_samples2=init_samples2.reset_index().iloc[:,1:]
    init_samples3 = sample_lab_attr_all(n_regions_list[2],3)
    init_samples3=init_samples3.reset_index().iloc[:,1:]

    init_samples=pd.concat([init_samples1,init_samples2,init_samples3],axis=0)"""
    ############
    
    #init_samples.head()
    init_samples=init_samples.reset_index()
    #init_samples.head()
    init_samples=init_samples.iloc[:,1:]
    #init_samples.index
    node_attr=init_samples
    node_attr['state']="high"

    node_attr['partition']=""
    for i in range(0,node_attr.shape[0]):
        if i<n_regions_list[0]:
            node_attr['partition'][i]='NrA'
        elif i< (n_regions_list[0]+n_regions_list[1]):
            node_attr['partition'][i]='Eur'
        else:
            node_attr['partition'][i]='Asia'

    #tier and MS merge with attributes
    node_attr = pd.concat([node_attr,nodes_frame.iloc[:,2:]],axis=1)
    #node_attr.columns

    node_attr.columns=['X1..Commitment...Governance', 'X2..Traceability.and.Risk.Assessment',
           'X3..Purchasing.Practices', 'X4..Recruitment', 'X5..Worker.Voice',
           'X6..Monitoring', 'X7..Remedy', 'score', 'state', 'partition','tier','ms','ms2']
    #

    #node_attr.info()



    # region wise reg assumption and market size assumption
    #p_reg_org=p_reg.copy()
    #p_med_org=p_med.copy()

    #
    init_node_attrs_df=node_attr.copy()
    init_edge_attrs_df=edge_list_df.copy()

    import os
    os.mkdir(folder_location+"sc_"+str(run_iter+1))


    node_attr.to_csv(folder_location+"sc_"+str(run_iter+1)+"/"+str(0)+ "_node_attr.csv")

    df_3=cosine_similarity(node_attr.iloc[:,:8])
    df_4=pd.DataFrame(df_3)
    df_4.values[[np.arange(len(df_4))]*2] = np.nan
    #mat_data.head()
    edge_list_2=df_4.stack().reset_index()
    edge_list_2.columns=['source','target','weight']
    #edge_list_2.head()
    edge_list_f=pd.merge(edge_list_df, edge_list_2,  how='left', left_on=['source','target'], right_on = ['source','target'])
    #edge_list_f.head()
    edge_list_f.drop('weight_x',axis=1,inplace=True)
    edge_list_f.columns=['source','target','weight']

    edge_list_f.to_csv(folder_location+"sc_"+str(run_iter+1)+"/"+"initial_edge_attr.csv")

    ##### run simulation for all

    print("@@@@@@@@@@@@@@@ -- "+str(run_iter))
    w1=scn_params.iloc[run_iter,5]
    w2=scn_params.iloc[run_iter,6]
    w3=scn_params.iloc[run_iter,7]
    w4=scn_params.iloc[run_iter,8]  
    w5=scn_params.iloc[run_iter,9]  
    
    r_on=scn_params.iloc[run_iter,10]
    m_on=scn_params.iloc[run_iter,11]
    alpha1=scn_params.iloc[run_iter,12]
    alpha2=scn_params.iloc[run_iter,13]
    alpha3=scn_params.iloc[run_iter,14]
    
    if N==500:
        num_sim=20
    else:
        num_sim=20
    probs_mat=np.zeros((num_sim,N))

    probs_mat2=np.zeros((((num_sim-1)*N)+1,N))

    ## Initial node and edge attributes

    node_attr=init_node_attrs_df.copy()
    edge_list_df=init_edge_attrs_df.copy()
    ################################################## simulation
    simulation_continous(node_attr=node_attr,edge_list_df=edge_list_df,num_sim=num_sim,w1=w1,w2=w2,w3=w3,w4=w4,w5=w5,bs1=bs1,bs2=bs2,N=N,r_on=r_on,m_on=m_on,p_reg=p_reg,p_med=p_med,probs_mat=probs_mat,probs_mat2=probs_mat2,run_iter=run_iter,alpha1=alpha1,alpha2=alpha2,alpha3=alpha3)

    lik_probs_mat=pd.DataFrame(probs_mat)
    lik_probs_mat2=pd.DataFrame(probs_mat2)
    
    lik_probs_mat.to_csv(folder_location+"sc_"+str(run_iter+1)+"/"+"lik_probs_mat.csv")
    lik_probs_mat2.to_csv(folder_location+"sc_"+str(run_iter+1)+"/"+"lik_probs_mat2.csv")
    del lik_probs_mat

    

In [ ]:
np.random.normal(50,5)

57.05878290395809

In [ ]:
# sample_lab_attr_all_init(np.float(10),30,10)

In [ ]:
from magic_functions import process_func

In [ ]:
frames_list = range(0,180)#range(0,8)

In [ ]:
max_pool=30

In [ ]:
with Pool(max_pool) as p:
    pool_outputs = list(
        tqdm(
            p.imap(process_func,
                   frames_list),
            total=len(frames_list)
        )
    )    


  0%|          | 0/180 [00:00<?, ?it/s]